In [1]:
import torch 
import torch.nn as nn
import torchvision.datasets as dsets
import torchvision.transforms as transforms
from torch.autograd import Variable

In [2]:
sequence_length = 28
input_size = 28
hidden_size = 128
num_layers = 2
num_classes = 10
batch_size = 100
num_epochs = 5
learning_rate = 0.01

In [3]:
train_dataset = dsets.MNIST(root='./data/',
                            train=True, 
                            transform=transforms.ToTensor(),
                            download=True)

test_dataset = dsets.MNIST(root='./data/',
                           train=False, 
                           transform=transforms.ToTensor())

# Data Loader (Input Pipeline)
train_loader = torch.utils.data.DataLoader(dataset=train_dataset,
                                           batch_size=batch_size, 
                                           shuffle=True)

test_loader = torch.utils.data.DataLoader(dataset=test_dataset,
                                          batch_size=batch_size, 
                                          shuffle=False)

In [4]:
class RNN(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, num_classes):
        super(RNN, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_size, num_classes)
    
    def forward(self, x):
        # Set initial states 
        h0 = Variable(torch.zeros(self.num_layers, x.size(0), self.hidden_size)) 
        c0 = Variable(torch.zeros(self.num_layers, x.size(0), self.hidden_size))
        
        # Forward propagate RNN
        out, _ = self.lstm(x, (h0, c0))  
        
        # Decode hidden state of last time step
        out = self.fc(out[:, -1, :])  
        return out
    

In [5]:
rnn = RNN(input_size, hidden_size,num_layers,num_classes)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(rnn.parameters(), lr=learning_rate)

In [6]:
for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(train_loader):
        images = Variable(images.view(-1, sequence_length, input_size))
        labels = Variable(labels)
        
        # Forward + Backward + Optimize
        optimizer.zero_grad()
        outputs = rnn(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        
        if (i+1) % 100 == 0:
            print ('Epoch [%d/%d], Step [%d/%d], Loss: %.4f' 
                   %(epoch+1, num_epochs, i+1, len(train_dataset)//batch_size, loss.data[0]))


Epoch [1/5], Step [100/600], Loss: 0.8334
Epoch [1/5], Step [200/600], Loss: 0.2995
Epoch [1/5], Step [300/600], Loss: 0.2101
Epoch [1/5], Step [400/600], Loss: 0.2151
Epoch [1/5], Step [500/600], Loss: 0.2279
Epoch [1/5], Step [600/600], Loss: 0.2093
Epoch [2/5], Step [100/600], Loss: 0.0433
Epoch [2/5], Step [200/600], Loss: 0.0961
Epoch [2/5], Step [300/600], Loss: 0.1230
Epoch [2/5], Step [400/600], Loss: 0.0958
Epoch [2/5], Step [500/600], Loss: 0.1155
Epoch [2/5], Step [600/600], Loss: 0.1161
Epoch [3/5], Step [100/600], Loss: 0.1399
Epoch [3/5], Step [200/600], Loss: 0.0860
Epoch [3/5], Step [300/600], Loss: 0.0520
Epoch [3/5], Step [400/600], Loss: 0.1304
Epoch [3/5], Step [500/600], Loss: 0.0120
Epoch [3/5], Step [600/600], Loss: 0.0641
Epoch [4/5], Step [100/600], Loss: 0.0413
Epoch [4/5], Step [200/600], Loss: 0.0839
Epoch [4/5], Step [300/600], Loss: 0.0178
Epoch [4/5], Step [400/600], Loss: 0.0661
Epoch [4/5], Step [500/600], Loss: 0.0765
Epoch [4/5], Step [600/600], Loss:

In [7]:
correct = 0
total = 0
for images, labels in test_loader:
    images = Variable(images.view(-1, sequence_length, input_size))
    outputs = rnn(images)
    _, predicted = torch.max(outputs.data, 1)
    total += labels.size(0)
    correct += (predicted == labels).sum()

print('Test Accuracy of the model on the 10000 test images: %d %%' % (100 * correct / total)) 

# Save the Model
torch.save(rnn.state_dict(), 'rnn.pkl')

Test Accuracy of the model on the 10000 test images: 98 %


In [8]:
class BiRNN(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, num_classes):
        super(BiRNN, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, 
                            batch_first=True, bidirectional=True)
        self.fc = nn.Linear(hidden_size*2, num_classes)  # 2 for bidirection 
    
    def forward(self, x):
        # Set initial states
        h0 = Variable(torch.zeros(self.num_layers*2, x.size(0), self.hidden_size)) # 2 for bidirection 
        c0 = Variable(torch.zeros(self.num_layers*2, x.size(0), self.hidden_size))
        
        # Forward propagate RNN
        out, _ = self.lstm(x, (h0, c0))
        
        # Decode hidden state of last time step
        out = self.fc(out[:, -1, :])
        return out


In [9]:
rnn = BiRNN(input_size, hidden_size, num_layers, num_classes)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(rnn.parameters(), lr=learning_rate)

In [10]:
for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(train_loader):
        images = Variable(images.view(-1, sequence_length, input_size))
        labels = Variable(labels)
        
        # Forward + Backward + Optimize
        optimizer.zero_grad()
        outputs = rnn(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        
        if (i+1) % 100 == 0:
            print ('Epoch [%d/%d], Step [%d/%d], Loss: %.4f' 
                   %(epoch+1, num_epochs, i+1, len(train_dataset)//batch_size, loss.data[0]))


Epoch [1/5], Step [100/600], Loss: 1.0872
Epoch [1/5], Step [200/600], Loss: 0.4915
Epoch [1/5], Step [300/600], Loss: 0.2502
Epoch [1/5], Step [400/600], Loss: 0.3389
Epoch [1/5], Step [500/600], Loss: 0.1265
Epoch [1/5], Step [600/600], Loss: 0.1251
Epoch [2/5], Step [100/600], Loss: 0.1953
Epoch [2/5], Step [200/600], Loss: 0.1491
Epoch [2/5], Step [300/600], Loss: 0.0701
Epoch [2/5], Step [400/600], Loss: 0.0552
Epoch [2/5], Step [500/600], Loss: 0.0999
Epoch [2/5], Step [600/600], Loss: 0.1628
Epoch [3/5], Step [100/600], Loss: 0.1770
Epoch [3/5], Step [200/600], Loss: 0.1790
Epoch [3/5], Step [300/600], Loss: 0.1097
Epoch [3/5], Step [400/600], Loss: 0.1167
Epoch [3/5], Step [500/600], Loss: 0.0101
Epoch [3/5], Step [600/600], Loss: 0.0514
Epoch [4/5], Step [100/600], Loss: 0.0864
Epoch [4/5], Step [200/600], Loss: 0.0967
Epoch [4/5], Step [300/600], Loss: 0.0793
Epoch [4/5], Step [400/600], Loss: 0.0895
Epoch [4/5], Step [500/600], Loss: 0.1764
Epoch [4/5], Step [600/600], Loss:

In [11]:
correct = 0
total = 0
for images, labels in test_loader:
    images = Variable(images.view(-1, sequence_length, input_size))
    outputs = rnn(images)
    _, predicted = torch.max(outputs.data, 1)
    total += labels.size(0)
    correct += (predicted == labels).sum()

print('Test Accuracy of the model on the 10000 test images: %d %%' % (100 * correct / total)) 

# Save the Model
torch.save(rnn.state_dict(), 'BIrnn.pkl')

Test Accuracy of the model on the 10000 test images: 98 %
